# Initialize cluster and Moving randomly any molecule
# avoiding that any atoms overlaps each other:

In [1]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'amcess' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
def visualize(
        xyz_file: str, 
        sphere_radius: int, 
        sphere_center: tuple,
        style='sphere',
        ):
        """
        Visualize the molecule with a sphere around the molecule.
        """

        sr = sphere_radius
        cx, cy, cz = sphere_center

        if not xyz_file:
                return None

        # starting visualization
        xyz_view = py3Dmol.view() # width=400, height=300)#, linked=False, viewergrid=(2,2))
        xyz_view.addModelsAsFrames(xyz_file,'xyz')
        # xyz_view.setStyle({style: {'radius': 0.6}})

        if style == 'stick':
                xyz_view.setStyle({'stick': {'radius': 0.15}})
        else:
                xyz_view.setStyle({'sphere': {'radius': 0.8}})


        xyz_view.zoomTo()

        # animation base on several XYZ coordinates snapshots
        xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 1})

        # sphere center
        sc = {"x": cx, "y": cy, "z": cz}

        # 20% to check visually that everything is inside
        xyz_view.addSphere({'center': sc, 
                'radius': sr, 
                'color' :'yellow',
                'alpha': 0.5,
                })

        # cartesian 3D axes
        x_axis = {'start': {'x': cx-sr, 'y':cy, 'z':cz}, 'end': {'x': cx+sr, 'y':cy, 'z':cz}}
        y_axis = {'start': {'x': cx, 'y':cy-sr, 'z':cz}, 'end': {'x': cx, 'y':cy+sr, 'z':cz}}
        z_axis = {'start': {'x': cx, 'y':cy, 'z':cz-sr}, 'end': {'x': cx, 'y':cy, 'z':cz+sr}}

        xyz_view.addLine(x_axis)
        xyz_view.addLine(y_axis)
        xyz_view.addLine(z_axis)

        xyz_view.addLabel("x", {
                'position':x_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })
        xyz_view.addLabel("y", {
                'position':y_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })
        xyz_view.addLabel("z", {
                'position':z_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })

        return xyz_view

# xyz_view.show()

In [3]:
# py3Dmol: a simple IPython/Jupyter widget to embed an interactive 
# 3Dmol.js viewer in a notebook.
import py3Dmol

# importing de Molecule Class
from amcess.molecule import Molecule
from amcess.cluster import Cluster

In [4]:
hf = {"atoms": [("H", 0, 0, 0), ("F", 1, 0, 0)]}
cc = {"atoms": [("C", 0, 0, 0), ("C", 1.2, 0, 0)]}
ab = Cluster(hf, cc)

In [5]:
xyz_data = ab.GetBlockXYZ().replace('\t', '').split('\n')
xyz_data = xyz_data[2:]
xyz_data = ';'.join(xyz_data)

print(xyz_data)

H          0.00000000     0.00000000     0.00000000;F          1.00000000     0.00000000     0.00000000;C          0.00000000     0.00000000     0.00000000;C          1.20000000     0.00000000     0.00000000;


In [6]:
ab_xyz = ""

ab.SetSphereR(4)
# ab.freeze_molecule = 0

for _ in range(100):
    mol = ab.GetRandomGen().choice(ab.GetTotalMol())

    ab = ab.MoveMol(
        molecule=mol,
        max_step=3,
        max_rotation=10,
        max_closeness=0.5,
    )
    # print(ab.xyz)
    ab_xyz += ab.GetBlockXYZ()

# file_name = "cluster_test.xyz"
# with open(file_name, "w") as f:
#     f.write(ab_xyz)

In [7]:
view = visualize(        
    xyz_file = ab_xyz,
    sphere_radius = ab.GetSphereR() * 1.2,
    sphere_center = ab.GetSphereCenter(),
)
view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
ab_xyz = ""
ab.SetSphereR(4)
ab._freeze_molecule = None

In [9]:
ab.SetFreezeMol(0)

for _ in range(100):
    mol = ab.GetRandomGen().choice(ab.GetTotalMol())

    ab = ab.MoveMol(
        molecule=mol,
        max_step=4.0,
        max_rotation=30,
        max_closeness=1.0,
    )
    # print(ab.xyz)
    ab_xyz += ab.GetBlockXYZ()

# file_name = "cluster_test.xyz"
# with open(file_name, "w") as f:
#     f.write(ab_xyz)

# print(ab.freeze_molecule)
# print(ab.cluster_dictionary)

view = visualize(        
    xyz_file = ab_xyz,
    sphere_radius = ab.GetSphereR() * 1.2,
    sphere_center = ab.GetSphereCenter(),
)
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
import numpy as np

def overlapping(
    first_coordinates: list,
    second_coordinates: list,
    max_closeness: float = 1.0,
) -> bool:
    for first_atom in first_coordinates:
        for second_atom in second_coordinates:
            distance = np.linalg.norm(
                np.asarray(first_atom) - np.asarray(second_atom)
            )

            if distance < max_closeness:
                return True

    return False

In [11]:

def initialize_cluster(mol, max_closeness: float = 1.0) -> object:
    # center of mass coordinates
    sc_x = mol.GetSphereCenter()[0]
    sc_y = mol.GetSphereCenter()[1]
    sc_z = mol.GetSphereCenter()[2]

    # initializing a new cluster moving the first molecule
    # to the center of the cluster sphere
    molecule = mol.GetMol(0)
    new_cluster = Cluster(molecule).TranslateMol(0, sc_x, sc_y, sc_z)

    for i in range(1, mol.GetTotalMol()):
        # moving the next single molecule into the cluster sphere
        molecule = Cluster(mol.GetMol(i)).TranslateMol(0, sc_x, sc_y, sc_z)

        if overlapping(molecule.GetAtomicCoordinates(), new_cluster.GetAtomicCoordinates()):
            new_cluster += molecule
            new_cluster = new_cluster.MoveMol(
                i,
                max_step=None,
                max_rotation=None,
                max_closeness=max_closeness,
            )
        else:
            new_cluster += molecule

    return Cluster(
        new_cluster,
        freeze_molecule=mol.GetFreezeMol(),
        sphere_radius=mol.GetSphereR(),
        sphere_center=mol.GetSphereCenter(),
    )

In [12]:
hf6 = 6 * Cluster(hf)

hf6.SetSphereR(4)
hf6.SetSphereCenter((10, 10, 10))

print(hf6.GetSeed())

204464


In [13]:
hf6.SetSeed(932570)

new_hf6 =  initialize_cluster(hf6, max_closeness=2.0)
print(new_hf6.GetBlockXYZ())

12
charge: 0 multiplicity:1
H     	    10.00000000	    10.00000000	    10.00000000
F     	    11.00000000	    10.00000000	    10.00000000
H     	    11.54133092	     7.69911431	     8.84488121
F     	    12.48754483	     7.97688610	     9.01077895
H     	     8.60846642	     8.67378624	     8.08212764
F     	     9.49908328	     8.86105111	     7.66772026
H     	    11.78077232	    12.43672727	     9.40948680
F     	    12.74715631	    12.56390101	     9.63293440
H     	     8.25879300	    11.66143321	     7.96796371
F     	     9.24501504	    11.56155307	     7.83609239
H     	    11.14809031	    11.96474072	    12.18207079
F     	    12.09163048	    11.87919787	    11.86204835



In [14]:
new_view = visualize(
    xyz_file = new_hf6.GetBlockXYZ(),
    sphere_radius = new_hf6.GetSphereR() * 1.2,
    sphere_center = new_hf6.GetSphereCenter(),
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [15]:
water_molecule=[
    ("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)
    ]

w2 = Cluster(water_molecule, water_molecule)

# the water dimer was initialized with the same water molecule, so
# both molecules has the same coordinates an those atoms overlap each other
print(w2.GetBlockXYZ())
print(w2)

6
charge: 0 multiplicity:1
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000

Cluster of (2) molecules and (6) total atoms
 #0: molecule with 3 atoms:
     --> atoms: [('O', 0.0, 0.0, 0.0), ('H', 0.58708, 0.75754, 0.0), ('H', -0.58708, 0.75754, 0.0)]
     --> charge: +0
     --> multiplicity: 1
 #1: molecule with 3 atoms:
     --> atoms: [('O', 0.0, 0.0, 0.0), ('H', 0.58708, 0.75754, 0.0), ('H', -0.58708, 0.75754, 0.0)]
     --> charge: +0
     --> multiplicity: 1



In [16]:
#let's move any molecule randomly to avoid averlaping
new_w2 = w2.MoveMol(
    molecule = 1,
    max_step = 1.4,
    max_rotation = 30,
    max_closeness = 1.2,
)
print(new_w2.GetBlockXYZ())
print(new_w2)

6
charge: 0 multiplicity:1
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.42890042	     0.83651254	     1.35299848
H     	     1.12786931	     1.40308345	     1.68310405
H     	     0.16582476	     1.71945948	     1.08894229

Cluster of (2) molecules and (6) total atoms
 #0: molecule with 3 atoms:
     --> atoms: [('O', 0.0, 0.0, 0.0), ('H', 0.58708, 0.75754, 0.0), ('H', -0.58708, 0.75754, 0.0)]
     --> charge: +0
     --> multiplicity: 1
 #1: molecule with 3 atoms:
     --> atoms: [('O', 0.42890042, 0.83651254, 1.35299848), ('H', 1.12786931, 1.40308345, 1.68310405), ('H', 0.16582476, 1.71945948, 1.08894229)]
     --> charge: +0
     --> multiplicity: 1



In [17]:
#! Why eliminate one water molecule?
print(w2.GetBlockXYZ())

6
charge: 0 multiplicity:1
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000



In [18]:
print(w2)

Cluster of (2) molecules and (6) total atoms
 #0: molecule with 3 atoms:
     --> atoms: [('O', 0.0, 0.0, 0.0), ('H', 0.58708, 0.75754, 0.0), ('H', -0.58708, 0.75754, 0.0)]
     --> charge: +0
     --> multiplicity: 1
 #1: molecule with 3 atoms:
     --> atoms: [('O', 0.42890042, 0.83651254, 1.35299848), ('H', 1.12786931, 1.40308345, 1.68310405), ('H', 0.16582476, 1.71945948, 1.08894229)]
     --> charge: +0
     --> multiplicity: 1



In [19]:
from copy import deepcopy
from numpy import random
random_gen = random.default_rng(1234)

# freeze ibuprofen molecule to avoid be moved (just water molecules dancing)
# w2.freeze_molecule = 0

w2.SetSphereR(4)

new_w2 = w2
new_w2 = new_w2.InitializeCluster(max_closeness=1)

# to save snapshot and show as a movie
w2_xyz = ""

total_steps = 100

for i in range(total_steps):
    # # molecule to be selected between [0, w.total_molecules]
    # # freeze molecules will not be moved or rotated
    mol = random_gen.choice(new_w2.GetTotalMol())

    print(new_w2.GetBlockXYZ())


    # saving coordinates as a string
    w2_xyz += new_w2.GetBlockXYZ()
    # if i % 10 == 0: 
    #     w2_xyz += new_w2.xyz

    #----------------------------------------------------------

    new_w2 = new_w2.MoveMol(
        molecule=mol,
        max_step=2,
        max_rotation=30,
        max_closeness=0.5,
    )

# # file_name = "w2.xyz"
# # with open(file_name, "w") as f:
# #     f.write(w2_xyz)

6
charge: 0 multiplicity:1
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.42890042	     0.83651254	     1.35299848
H     	     1.12786931	     1.40308345	     1.68310405
H     	     0.16582476	     1.71945948	     1.08894229

6
charge: 0 multiplicity:1
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	    -1.56280437	     0.04001279	    -0.60294190
H     	    -0.94758457	     0.75721011	    -0.44274706
H     	    -2.03015258	     0.82512089	    -0.89228719

6
charge: 0 multiplicity:1
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	    -1.32962871	    -0.27326351	    -2.56198404
H     	    -0.67445522	     0.3357396

In [20]:
new_view = visualize(
    xyz_file = w2_xyz,
    sphere_radius =w2.GetSphereR() * 1.2,
    sphere_center = w2.GetSphereCenter(),
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [21]:
# for several molecules we could move randomly one-by-one
# or we could just `initialize` this cluster with all its molecules 
# moved around avoiding overlapping
w6 = 3 * w2

# let's define the spherical boundary conditions
w6.SetSphereR(10)

# maximum closeness is the minimum distance for two atoms
new_w6 = w6.InitializeCluster(max_closeness=3)

print(new_w6.GetBlockXYZ())

18
charge: 0 multiplicity:1
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	    -3.59702707	    -2.63685258	    -0.95054770
H     	    -2.89417615	    -2.06127467	    -0.64519472
H     	    -4.01616088	    -1.78104185	    -0.84834873
O     	    -3.59702707	    -2.63685258	    -0.95054770
H     	    -2.89417615	    -2.06127467	    -0.64519472
H     	    -4.01616088	    -1.78104185	    -0.84834873
O     	     5.62368836	     4.55036248	     0.09893623
H     	     6.36391470	     5.08087420	     0.39751963
H     	     5.30674625	     5.45339445	     0.04784088
O     	     5.62368836	     4.55036248	     0.09893623
H     	     6.36391470	     5.08087420	     0.39751963
H     	     5.30674625	     5.4533944

!pip install py3Dmol
import py3Dmol

In [22]:
new_view = visualize(
    xyz_file = new_w6.GetBlockXYZ(),
    sphere_radius =new_w6.GetSphereR() * 1.2,
    sphere_center = new_w6.GetSphereCenter(),
)
new_view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [23]:
from data.molecules_coordinates import water, ibuprofen 

# let's create a molecular object for the ibuprofen
ibu = Cluster(ibuprofen["atoms"])

# let's create a molecular object for one water molecule
w = Cluster(water["atoms"])

In [24]:
w6_overlap = Cluster(6 * w)

# let's initialize a cluster of six water without overlapping
w6 = Cluster(6 * w).InitializeCluster(max_closeness=2)


print(w6.GetBlockXYZ())


18
charge: 0 multiplicity:1
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	    -1.93018687	    -2.33799966	    -2.31946018
H     	    -1.69326943	    -1.51838489	    -2.75607390
H     	    -2.68262353	    -1.76277297	    -2.17288899
O     	    -1.42322311	    -0.12807393	     1.68558057
H     	    -1.09663221	     0.68842467	     2.06662940
H     	    -2.12220859	     0.50749016	     1.52431261
O     	     2.21160597	    -0.81377086	    -1.21418502
H     	     3.00476155	    -0.29634267	    -1.06686881
H     	     1.95180305	     0.08573582	    -1.41892864
O     	    -2.62543558	    -2.33873093	     4.45042318
H     	    -2.09189793	    -1.54725594	     4.36419849
H     	    -3.26046058	    -1.63806034	     4.29443037
O     	     1.97388270	     1.84169331	     2.38788730
H     	     2.46032861	     2.66732680	     2.37273807
H     	     1.43370610	     2.4277989

In [25]:
new_view = visualize(
    xyz_file = w6.GetBlockXYZ(),
    sphere_radius = 5,
    sphere_center = w6.GetSphereCenter(),
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [26]:
ibu_w6 = Cluster(ibu + w6)

# let's define a sphere with center around the carboxylic acid group
ibu_w6.SetSphereCenter((-3, 0.2, 0.2))
ibu_w6.SetSphereR(4)

ibu_w6 = ibu_w6.InitializeCluster(max_closeness=2)

print(ibu_w6.GetBlockXYZ())

50
charge: 0 multiplicity:1
C     	    -1.56098300	    -1.10947200	     1.17219000
C     	    -2.94477900	    -1.24360900	     1.33412100
C     	    -3.84450500	    -0.79651100	     0.35585200
C     	    -3.30141000	    -0.20160700	    -0.79830200
C     	    -1.92211300	    -0.07098900	    -0.95773900
C     	    -1.02102000	    -0.52007300	     0.02281900
H     	    -0.89290600	    -1.46731500	     1.95548700
H     	    -3.33425300	    -1.70043400	     2.24251900
H     	    -3.98572600	     0.18365200	    -1.54865500
H     	    -1.53569500	     0.39530700	    -1.86436000
C     	    -5.34102100	    -0.97417500	     0.53451400
H     	    -5.51739900	    -1.32551300	     1.55744700
C     	     0.47407100	    -0.34878900	    -0.15756300
H     	     0.76256300	    -0.69849900	    -1.16041200
H     	     1.00698600	    -0.99284600	     0.55851800
C     	    -5.90009800	    -2.02131200	    -0.45068600
H     	    -5.37923800	    -2.98385700	    -0.34592400
H     	    -6.96972400	    -2.1840250

In [27]:
new_view = visualize(
    xyz_file = ibu_w6.GetBlockXYZ(),
    sphere_radius = ibu_w6.GetSphereR() * 1.2,
    sphere_center = ibu_w6.GetSphereCenter(),
    style='stick',
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [28]:
from numpy import random
random_gen = random.default_rng(1234)

new_ibu_w6 = ibu_w6

# to save snapshot and show as a movie
new_ibu_w6_xyz = ""

# freeze ibuprofen molecule to avoid be moved (just water molecules dancing)
# new_ibu_w6.freeze_molecule = 0

new_ibu_w6.SetSphereR(3)
new_ibu_w6.SetSphereCenter((-6.12410200, 0.37976300, 0.38520700))


# to avoid overlapping and any molecule out of the sphere
ibu_w6 = ibu_w6.InitializeCluster(max_closeness=2)

total_steps = 100

for i in range(total_steps):
    # # molecule to be selected between [0, w.total_molecules]
    # # freeze molecules will not be moved or rotated
    mol = random_gen.choice(new_ibu_w6.GetTotalMol())

    if mol == 0:
        continue

    # saving coordinates as a string
    new_ibu_w6_xyz += new_ibu_w6.GetBlockXYZ()
    # if i % 10 == 0: 
        # new_ibu_w6_xyz += new_ibu_w6.xyz

    # #----------------------------------------------------------

    new_ibu_w6 = new_ibu_w6.MoveMol(
        molecule=mol,
        max_closeness=0.5,
        max_step=2,
        max_rotation=30,    
    )

    #----------------------------------------------------------


    # printing current step
    print(
        f"\r progress {100.0*((i + 1)/total_steps):.2f}"
        f" % -- step {i + 1}/{total_steps}", end=''
        )
# -------------------------------------------------------
print("\n *** JOB DONE ***")
print(f"after {total_steps} steps\n")
# print(new_ibu_w6_xyz)

# open("new_ibu_w6.xyz", "w").write(new_ibu_w6_xyz)

 progress 99.00 % -- step 99/100
 *** JOB DONE ***
after 100 steps



In [29]:
new_view = visualize(
    xyz_file = new_ibu_w6_xyz,
    sphere_radius = new_ibu_w6.GetSphereR() * 1.2,
    sphere_center = new_ibu_w6.GetSphereCenter(),
    style='stick',
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol